In [8]:
import json
import pandas as pd
import matplotlib.pyplot as plt
from names_dataset import NameDataset

In [2]:
file_path = "../data/train.json"

with open(file_path, "r") as file:
    data = json.load(file)

df = pd.DataFrame(data)

In [3]:
df.head()

,document,full_text,tokens,trailing_whitespace,labels
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."


In [26]:
labels = df.labels.explode().reset_index(drop=True)
labels_vc = labels.value_counts()
labels_vc

labels
O                   4989794
B-NAME_STUDENT         1365
I-NAME_STUDENT         1096
B-URL_PERSONAL          110
B-ID_NUM                 78
B-EMAIL                  39
I-STREET_ADDRESS         20
I-PHONE_NUM              15
B-USERNAME                6
B-PHONE_NUM               6
B-STREET_ADDRESS          2
I-URL_PERSONAL            1
I-ID_NUM                  1
Name: count, dtype: int64

### B-ID_NUM

In [62]:
idx = labels[labels == "B-ID_NUM"].index
b_ids = df.tokens.explode().reset_index(drop=True)[idx]
b_ids.shape

(78,)

In [71]:
specific_id = b_ids.str.match(r'\d{6,12}\b|\b[a-zA-Z]+\d{2,}')
ids_in_tokens = df.tokens.explode().str.match(r'\d{6,12}\b|\b[a-zA-Z]+\d{2,}')

print('Regex finding Ids in label specific: ', specific_id.sum())
print('Regex finding Ids in all tokens: ', ids_in_tokens.sum())
print()
print('Accuracy Regex finding Ids in label specific: ', specific_id.sum() / len(specific_id))
# We calculate the percentage of ids we would have classified wrongly in the total amount of tokens
print('Error of missclassification in whole', ids_in_tokens.sum() / len(ids_in_tokens))

Regex finding Ids in label specific:  61
Regex finding Ids in all tokens:  186

Accuracy Regex finding Ids in label specific:  0.782051282051282
Error of missclassification in whole 3.725563756914576e-05


### B-EMAIL

In [73]:
idx = labels[labels == "B-EMAIL"].index
b_ids = df.tokens.explode().reset_index(drop=True)[idx]
b_ids.shape

(39,)

In [74]:
reg = r'[\w\.-]+@[\w\.-]+'
specific_id = b_ids.str.match(reg)
ids_in_tokens = df.tokens.explode().str.match(reg)

print('Regex finding Ids in label specific: ', specific_id.sum())
print('Regex finding Ids in all tokens: ', ids_in_tokens.sum())
print()
print('Accuracy Regex finding Ids in label specific: ', specific_id.sum() / len(specific_id))
# We calculate the percentage of ids we would have classified wrongly in the total amount of tokens
print('Error of missclassification in whole', ids_in_tokens.sum() / len(ids_in_tokens))

Regex finding Ids in label specific:  39
Regex finding Ids in all tokens:  49

Accuracy Regex finding Ids in label specific:  1.0
Error of missclassification in whole 9.814657209076034e-06


### B-URL_PERSONAL

### I-PHONE_NUM
**NOTE**: For this one it seems we need to do some sort of kernel implementation because multiple tokens can be part of the phone number. This includes the B-PHONE_NUM tag as well.

In [83]:
idx = labels[labels == "I-PHONE_NUM"].index
b_ids = df.tokens.explode().reset_index(drop=True)[idx]
b_ids

150910       320)202
150911             -
150912    0688x95843
179881       223)392
179882             -
179883          2765
286836       820)913
286837             -
286838      3241x894
287267       820)913
287268             -
287269      3241x894
287294       820)913
287295             -
287296      3241x894
Name: tokens, dtype: object

In [81]:
reg = r'\b\d{3}[-.]?\d{3}[-.]?\d{4}\b'
specific_id = b_ids.str.match(reg)
ids_in_tokens = df.tokens.explode().str.match(reg)

print('Regex finding Ids in label specific: ', specific_id.sum())
print('Regex finding Ids in all tokens: ', ids_in_tokens.sum())
print()
print('Accuracy Regex finding Ids in label specific: ', specific_id.sum() / len(specific_id))
# We calculate the percentage of ids we would have classified wrongly in the total amount of tokens
print('Error of missclassification in whole', ids_in_tokens.sum() / len(ids_in_tokens))

Regex finding Ids in label specific:  1
Regex finding Ids in all tokens:  1

Accuracy Regex finding Ids in label specific:  0.16666666666666666
Error of missclassification in whole 2.0029912671583744e-07


## Model Definition
We will attempt to create the simplest model possible. The model will evaluate each token and pass it through a couple functions to evaluate it. If one of the functions returns true, the token will be considered a keyword. The model will then return the keyword and the index of the token in the input string.

### Model Functions
- **email** This function will use regex to identify if the token is an email address.
- **phone_num** This function will use regex to identify if the token is a phone number.
- **address** This function will use regex to identify if the token is an address.
- **username** This function will use a word embedding model to identify if the token is a common word and if not then we flag it as a username.
- **personal_id** This function will use regex to identify if the token is a personal id.


In [ ]:
class PiiModel:
    def __init__(self, train_set: pd.DataFrame) -> None:
        self.train_set = train_set
        pass

    def get_name_dataset(self):
        return NameDataset()
    
    def get_email_regex(self):
        return r'[\w\.-]+@[\w\.-]+'
    
    def get_phone_number_regex(self):
        return r'\b\d{3}[-.]?\d{3}[-.]?\d{4}\b'
    
    def get_address_regex(self):
        return r'\d{1,5}\s\w.\s(\b\w*\b\s){1,2}\w*\.'
    
    def get_personal_id_regex(self):
        # Any 
        return r'\d{6,12}\b|\b[a-zA-Z]+\d{2,}'
    
    def get_url_personal_regex(self):
        return r'https?://[\w\.-]+'

